In [102]:
# Importamos con reglas de donde empieza la cabecera y si hay que cortar la cola de suicidios
# y renombramos columnas

import pandas as pd

files = [
    'data/Copia-de-femicidios_2010-1.xlsx',
    'data/Copia-de-femicidios_2011.xlsx',
    'data/Copia-de-femicidios_2012.xlsx',
    'data/Copia-de-femicidios_2013.xlsx',
    'data/Copia-de-femicidios2014final.xlsx',
    'data/Femicidios-2015-1-2.xlsx',
    'data/Femicidios-2016.xls',
    'data/Femicidios 2017 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2017.csv',
    'data/Femicidios 2018 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2018.csv',
    'data/Femicidios 2019 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2019.csv'
]
line_1_files = [
    'data/Femicidios-2016.xls',
    'data/Femicidios 2017 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2017.csv',
]
trim_files = [
    'data/Femicidios 2017 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2017.csv',
    'data/Femicidios 2019 - Red Chilena contra la Violencia hacia las Mujeres - FEMICIDIOS 2019.csv'
]
cols_rename = {
    'Antecedentes Judiciales': 'Antecedentes',
    'Antecedentes judiciales': 'Antecedentes',
    'Relación ': 'Relación',
    'Categoria Delito/Ley Femic.': 'Categoría delito/ Ley Femicidio',
    'Categoria Delito': 'Categoría delito/ Ley Femicidio',
    'Categoria Delito': 'Categoría delito/ Ley Femicidio',
    'Categoria delito ': 'Categoría delito/ Ley Femicidio',
    'Denuncia/ M cautelar': 'Denuncia/M cautelar',
    'Ocupación ': 'Ocupación femicida',
    'Edad ': 'Edad',
    'Ocupación.1': 'Ocupación femicida',
    'Edad.1': 'Edad femicida',
    'Edad .1': 'Edad femicida',
    'Edad 1': 'Edad femicida',
    'Antecedentes ': 'Antecedentes',
    'Fecha y Lugar': 'Fecha y lugar',
    'Femicidia': 'Femicida',
    'Nacionalidad / Etnia.1': 'Nacionalidad / Etnia femicida',
    'Unnamed: 24': 'Información medios',
    'Unnamed: 25': 'Información medios 2',
}
cols_delete = [
    'Unnamed: 15',
    'Unnamed: 16',
    'Unnamed: 0',
]
dataframes = []
for file in files:
    if file in line_1_files:
        header_line = 1
    else:
        header_line = 2
    if file.split('.')[-1] == 'csv':
        current_df = pd.read_csv(file, header=header_line)
    else:
        current_df = pd.read_excel(file, header=header_line, dtype={'Fecha y lugar': str})
    for column in current_df:
        for col_rename in cols_rename:
            if column == col_rename:
                current_df.rename(columns={column: cols_rename[column]}, inplace=True)
        for col_delete in cols_delete:
            if column == col_delete:
                del current_df[column]
    current_df['source'] = file
    if file in trim_files:
        current_df = current_df[:-3] #Cola de suicidios 
    dataframes.append(current_df)

In [103]:
df = pd.concat(dataframes, sort=False).reset_index(drop=True)

In [104]:
# TODO:
# - parse / make array references
# - check categorical values

In [158]:
def parse_fecha_lugar(row):
    if pd.notnull(row['Fecha y lugar']):
        # Casos particulares
        if row['Fecha y lugar'] == 'Collipulli12/12/2015':
            row['Fecha y lugar'] = '12/12/2015 Collipulli'
        if row['Fecha y lugar'] == '27/07Calama':
            row['Fecha y lugar'] = '27/07 Calama'
        if row['Fecha y lugar'] == '17/10PuertoMontt':
            row['Fecha y lugar'] = '17/10 PuertoMontt'
        if row['Fecha y lugar'] == 'Descubierto 25/09 Puerto Montt':
            row['Fecha y lugar'] = '25/09 Puerto Montt'
        if row['Fecha y lugar'] == '2015-11-21':
            row['Fecha y lugar'] = '21/11'
            
        
        
        # Para capturar la fecha
        spacers = ['.', '/ ','/',', ',',']
        for spacer in spacers:
            if row['Fecha y lugar'][:5].find(spacer) > 0:
                tmp_date = {}
                day = row['Fecha y lugar'][:5].split(spacer)[0]
                month = row['Fecha y lugar'][:5].split(spacer)[1][:2].strip()
                if (len(month) > 1):
                    if not month[1].isdigit():
                        print('Triming month: '+ month)
                        month = month[0]
                tmp = [int(s) for s in row['source'] if s.isdigit()][:4]
                year = "".join(str(x) for x in tmp)
                try:
                    row['Fecha'] = pd.to_datetime(int(year) * 10000 + int(month) * 100 + int(day),format='%Y%m%d')
                except:
                    print('Failed date:')
                    print(year, month, day)
                #print(row['Fecha y lugar'])
                
        dividers = [' - ', '- ', ' ', '-']   
        # Para capturar el lugar
        found_lugar = False
        for divider in dividers:
            if (row['Fecha y lugar'].find(divider) > 0) and (not found_lugar):
                row['Lugar'] = " ".join(row['Fecha y lugar'].split(divider)[1:])
                found_lugar = True
                
        if not found_lugar:
            print('Not able to parse place:')
            print(row['Fecha y lugar'])
    else:
        row['Fecha y lugar'] = pd.to_datetime(row['Fecha y lugar'])
            
    return row
#bar = df.sample(n=100).apply(parse_fecha_lugar, axis=1)
df_parsed = df.apply(parse_fecha_lugar, axis=1)

Triming month: 1-
Not able to parse place:
27/03
Failed date:
2013 0 20
Failed date:
2013 0 20
Not able to parse place:
21/04
Not able to parse place:
22/10


In [159]:
df_parsed.rename(columns={
    'Fecha y lugar': 'old_fecha_y_lugar',
    'Lugar': 'old_lugar',
    'Región': 'old_region'
}, inplace=True)

In [160]:
lugar_replacements = {
    'Valpo': 'Valparaíso',
    'Valparaiso': 'Valparaíso',
    'Buín': 'Buin',
    'Conchali': 'Conchalí',
    'San Bdo': 'San Bernardo',
    'Los Angeles': 'Los Ángeles',
    'Santiago Centro': 'Santiago',
    'Pto Montt': 'Puerto Montt',
    'Pto. Montt': 'Puerto Montt',
    'PuertoMontt': 'Puerto Montt',
    'Labranza': 'Temuco',
    'Padre Las Casas': 'Padre las Casas',
    'Padre Casas': 'Padre las Casas',
    'Tal Tal': 'Taltal',
    'Con-con': 'Concón',
    'Lontué': 'Molina',
    'Entre Lagos': 'Puyehue',
    'Est. Central': 'Estación Central',
    'Llolleo, San Antonio': 'San Antonio',
    'Pte Alto': 'Puente Alto',
    'Pte. Alto': 'Puente Alto',
    'Lautaro Temuco': 'Lautaro',
    'Batuco': 'Lampa',
    'Bucalemu, Paredones': 'Paredones',
    'Carrizal, San Javier': 'San Javier',
    'Cisterna': 'La Cisterna',
    'Cohihueco': 'Coihueco',
    'Dichato, Tomé': 'Tomé',
    'El Olivar': 'Olivar',
    'Hualaihue': 'Hualaihué',
    'Huentelolen': 'Cañete',
    'Indpenedencia': 'Independencia',
    'Lanco Los Lagos': 'Los Lagos',
    'LimacheV': 'Limache',
    'Liquiñe': 'Panguipulli',
    'Longavì': 'Longaví',
    'LosAndes': 'Los Andes',
    'Macúl': 'Macul',
    'Neltume': 'Panguipulli',
    'PAC': 'Pedro Aguirre Cerda',
    'Pemuco/Ñuble': 'Ñuble',
    'Peñalolen': 'Peñalolén',
    'Pitrufquen': 'Pitrufquén',
    'Pta.Arenas': 'Punta Arenas',
    'Pto. Aysen': 'Aysén',
    'Puerto Aysén': 'Aysén',
    'Puero Varas': 'Puerto Varas',
    'Puerto Cisnes': 'Cisnes',
    'Quellon': 'Quellón',
    'Quilpue': 'Quilpué',
    'Quinquehua': 'San Carlos',
    'Rallenco': 'Tirúa',
    'Rapel': 'Navidad',
    'Reinaco': 'Renaico',
    'Romero, Serena': 'La Serena',
    'San Francisco de Mostazal': 'Mostazal',
    'San José de la Mariquina': 'Mariquina',
    'San Pedro d/Paz': 'San Pedro de la Paz',
    'San Vicente de Tagua Tagua': 'San Vicente',
    '11 San Miguel': 'San Miguel', # From df1.apply(check_comuna, axis=1) => format issue
    '11 La Ligua': 'La Ligua', # format issue
    '11 San Miguel': 'San Miguel', # format issue
    'Joaquín': 'San Joaquín', # No date !!!!! => Rosa Elena Leletier López 2012
    '04 San Bernardo': 'San Bernardo', # format issue
    'Alto': 'Puente Alto', # format issue,
    'Paipote,': 'Tierra Amarilla', # format issue,
    'El Salvador': 'Diego de Almagro',
    'Pemuco/Ñuble': 'Pemuco',
    '00:00:00': None, # Null values,
    '17/12/2015': None, # Null values,
    '21/12/2015': None, # Null values,
    '22/12/2015': None, # Null values,
    '26/12/2015': None, # Null values
    'Araucanía': None, # Null values
    'RM': 'Buin', # Esta al revéz (comuna en lugar de región)
}
def fix_comuna(comuna):
    if type(comuna) == str:
        comuna = comuna.strip()
    if comuna in lugar_replacements:
        return lugar_replacements[comuna]
    else:
        return comuna
df_parsed['comuna'] = df_parsed.old_lugar.apply(fix_comuna)

In [161]:
# Check comunas
comunas = pd.read_csv('data/BDCUT_CL__CSV_UTF8.csv').COMUNA_NOMBRE
def check_comuna(row):
    #print(row)
    comuna = row['comuna']
    if type(comuna) == str:
        comuna = comuna.strip()
        if comuna not in comunas.values:
            return 'Error @ {} => {}'.format(row['Lugar'], comuna)
df_parsed.apply(check_comuna, axis=1).dropna()

Series([], dtype: object)

In [162]:
regiones = pd.read_csv('data/BDCUT_CL__CSV_UTF8.csv').REGION_NOMBRE

def fix_region(region):
    region_replacements = {
        'I': 'Tarapacá',
        'II': 'Antofagasta',
        'III': 'Atacama',
        'III Atacama': 'Atacama',
        'IV': 'Coquimbo',
        'V': 'Valparaíso',
        'Cabildo': 'Valparaíso',
        'VI': 'Región del Libertador Gral. Bernardo O\'Higgins',
        'O\'Higgins': 'Región del Libertador Gral. Bernardo O\'Higgins',
        'O’Higgins': 'Región del Libertador Gral. Bernardo O\'Higgins',
        'VII': 'Región del Maule',
        'VII Maule': 'Región del Maule',
        'Maule': 'Región del Maule',
        'El Maule': 'Región del Maule',
        'VIII': 'Región del Biobío',
        'Bio Bio': 'Región del Biobío',
        'Bío Bío': 'Región del Biobío',
        'Bío-Bío': 'Región del Biobío',
        'Ñuble': 'Región de Ñuble',
        'IX': 'Región de la Araucanía',
        'Araucanía': 'Región de la Araucanía',
        'IX Araucanía': 'Región de la Araucanía',
        'La Araucanía': 'Región de la Araucanía',
        'X': 'Región de Los Lagos',
        'Los Lagos': 'Región de Los Lagos',
        'XI': 'Atacama',
        'XII': 'Región Aisén del Gral. Carlos Ibáñez del Campo',
        'Aysen': 'Región Aisén del Gral. Carlos Ibáñez del Campo',
        'RM': 'Región Metropolitana de Santiago',
        'Rm': 'Región Metropolitana de Santiago',
        'Metropolitana': 'Región Metropolitana de Santiago',
        'Buin': 'Región Metropolitana de Santiago',
        'XIV': 'Región de Los Ríos',
        'Los Rios': 'Región de Los Ríos',
        'Los Ríos': 'Región de Los Ríos',
        'XV': 'Arica y Parinacota',
    }
    for region_replacement in region_replacements:
        if str(region).strip() == region_replacement:
            return region_replacements[region_replacement]
    return region
df_parsed['Region'] = df_parsed.old_region.apply(fix_region)

In [163]:
# Check region (15 + None = 16)
df_parsed.Region.unique().shape

(16,)

In [164]:
for region in df1.Región.apply(fix_region).unique():
    if region not in regiones.unique():
        print('Missing: {}'.format(region))

Missing: nan


In [165]:
df_parsed.columns.values

array(['old_fecha_y_lugar', 'old_region', 'Nombre', 'Edad', 'Ocupación',
       'Hecho', 'Vsexual', 'Tipo Femicidio', 'Femicida', 'Edad femicida',
       'Relación', 'Suicidio', 'Ocupación femicida', 'Antecedentes',
       'Denuncia/M cautelar', 'Categoría delito/ Ley Femicidio',
       'Sentencia', 'source', 'Sernam', 'Situación judicial', 'Tribunal',
       'Prensa', 'Fecha', 'old_lugar', 'Nacionalidad / Etnia',
       'Nacionalidad / Etnia femicida', 'Información medios',
       'Información medios 2', 'comuna', 'Region'], dtype=object)

In [170]:
# export with col order
col_order = [
    'old_fecha_y_lugar',
    'Fecha',
    'comuna',
    'old_region',
    'Region',
    'Nombre',
    'Edad',
    'Ocupación',
    'Hecho',
    'Vsexual',
    'Tipo Femicidio',
    'Femicida',
    'Edad femicida',
    'Relación',
    'Suicidio',
    'Ocupación femicida',
    'Antecedentes',
    'Denuncia/M cautelar',
    'Categoría delito/ Ley Femicidio',
    'Sentencia',
    'source',
    'Sernam',
    'Situación judicial',
    'Tribunal',
    'Prensa',
    'Nacionalidad / Etnia',
    'Nacionalidad / Etnia femicida',
    'Información medios',
    'Información medios 2'
]
df_parsed[col_order].to_csv('preview.csv', index=False)

In [167]:
# Count fields
from collections import Counter

counter = Counter()
for dataframe in dataframes:
    counter.update(dataframe.columns)
counter

Counter({'Fecha y lugar': 7,
         'Región': 10,
         'Nombre': 10,
         'Edad': 10,
         'Ocupación': 10,
         'Hecho': 10,
         'Vsexual': 10,
         'Tipo Femicidio': 10,
         'Femicida': 10,
         'Edad femicida': 10,
         'Relación': 10,
         'Suicidio': 10,
         'Ocupación femicida': 10,
         'Antecedentes': 10,
         'Denuncia/M cautelar': 10,
         'Categoría delito/ Ley Femicidio': 10,
         'Sentencia': 10,
         'source': 10,
         'Sernam': 8,
         'Situación judicial': 7,
         'Tribunal': 7,
         'Prensa': 1,
         'Fecha': 3,
         'Lugar': 3,
         'Nacionalidad / Etnia': 3,
         'Nacionalidad / Etnia femicida': 3,
         'Información medios': 3,
         'Información medios 2': 3})

In [168]:
# Check cols with same name
def duplicated_varnames(df):
    """Return a dict of all variable names that 
    are duplicated in a given dataframe."""
    repeat_dict = {}
    var_list = list(df) # list of varnames as strings
    for varname in var_list:
        # make a list of all instances of that varname
        test_list = [v for v in var_list if v == varname] 
        # if more than one instance, report duplications in repeat_dict
        if len(test_list) > 1: 
            repeat_dict[varname] = len(test_list)
    return repeat_dict

for dataframe in dataframes:
    print(duplicated_varnames(dataframe))

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
